# R4 Mark B

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

2022-10-20 21:22:10.489481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-20 21:22:10.821040: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-20 21:22:10.821118: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-20 21:22:10.879852: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-20 21:22:12.674573: W tensorflow/stream_executor/platform/de

In [12]:
df = pd.read_csv('../exportedDataframe/a_features.csv')

In [13]:
def read_dataframe(dataframe):
    dfc = dataframe.copy(True)
    dfc.drop('ImagesName', inplace=True, axis=1)
    return dfc
    

In [14]:
dfc = read_dataframe(df)

In [15]:
dfc

,CDR,VCDR,RDR,I,S,N,T,Labels
0,0.543767,0.442765,0.116711,220,205,131,140,0
1,0.410188,0.344595,0.088472,193,226,112,188,0
2,0.431755,0.389447,0.231198,169,151,130,128,0
3,0.388740,0.375648,0.131367,171,173,109,180,0
4,0.408971,0.388471,0.147757,169,188,123,173,0
...,...,...,...,...,...,...,...,...
163,0.424157,0.399471,0.235955,149,144,133,122,1
164,0.447977,0.487421,0.167630,125,121,126,141,1
165,0.426710,0.454861,0.166124,120,101,110,122,1
166,0.345794,0.385417,0.233645,110,114,140,108,1


## Trial 1: StandardScaler
dengan RDR max 0.19

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
from sklearn.preprocessing import StandardScaler

In [19]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=30)

2022-10-20 21:34:22.957688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-20 21:34:22.957777: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-20 21:34:22.957846: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2022-10-20 21:34:22.960568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/tsdhrm/.local/share/virtualenvs/ayoluluss2-CxxF7UsW/lib/python3.10/site-packages/keras/optimizers/optimizer_v2

Epoch 1/30
5/5 [==============================] - 1s 4ms/step - loss: 0.7254 - accuracy: 0.5075 - precision: 0.5323 - recall: 0.4714
Epoch 2/30
5/5 [==============================] - 0s 3ms/step - loss: 0.6638 - accuracy: 0.6343 - precision: 0.5946 - recall: 0.9429
Epoch 3/30
5/5 [==============================] - 0s 4ms/step - loss: 0.6338 - accuracy: 0.6343 - precision: 0.6438 - recall: 0.6714
Epoch 4/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5987 - accuracy: 0.6642 - precision: 0.6923 - recall: 0.6429
Epoch 5/30
5/5 [==============================] - 0s 3ms/step - loss: 0.5789 - accuracy: 0.6642 - precision: 0.6866 - recall: 0.6571
Epoch 6/30
5/5 [==============================] - 0s 3ms/step - loss: 0.5668 - accuracy: 0.7313 - precision: 0.8400 - recall: 0.6000
Epoch 7/30
5/5 [==============================] - 0s 4ms/step - loss: 0.5643 - accuracy: 0.7537 - precision: 0.8936 - recall: 0.6000
Epoch 8/30
5/5 [==============================] - 0s 3ms/step - loss:

In [23]:
predictions = model.predict(X_test_scaled)

2/2 [==============================] - 0s 3ms/step


In [24]:
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.82
Precision: 0.75
Recall: 0.86


## Trial 2: MinMaxScaler

In [26]:
df = pd.read_csv('../exportedDataframe/a_features.csv')
dfc = read_dataframe(df)

In [27]:
dfc.head(5)

,CDR,VCDR,RDR,I,S,N,T,Labels
0,0.543767,0.442765,0.116711,220,205,131,140,0
1,0.410188,0.344595,0.088472,193,226,112,188,0
2,0.431755,0.389447,0.231198,169,151,130,128,0
3,0.388740,0.375648,0.131367,171,173,109,180,0
4,0.408971,0.388471,0.147757,169,188,123,173,0


In [28]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [29]:
from sklearn.preprocessing import MinMaxScaler

In [30]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30


/home/tsdhrm/.local/share/virtualenvs/ayoluluss2-CxxF7UsW/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


5/5 [==============================] - 1s 2ms/step - loss: 0.8237 - accuracy: 0.4701 - precision: 0.4912 - recall: 0.4000    
Epoch 2/30
5/5 [==============================] - 0s 3ms/step - loss: 0.6946 - accuracy: 0.5224 - precision: 0.5224 - recall: 1.0000
Epoch 3/30
5/5 [==============================] - 0s 4ms/step - loss: 0.6940 - accuracy: 0.5224 - precision: 0.5224 - recall: 1.0000
Epoch 4/30
5/5 [==============================] - 0s 4ms/step - loss: 0.6925 - accuracy: 0.5224 - precision: 0.5224 - recall: 1.0000
Epoch 5/30
5/5 [==============================] - 0s 3ms/step - loss: 0.6921 - accuracy: 0.5224 - precision: 0.5224 - recall: 1.0000
Epoch 6/30
5/5 [==============================] - 0s 4ms/step - loss: 0.6956 - accuracy: 0.4627 - precision: 0.4902 - recall: 0.7143
Epoch 7/30
5/5 [==============================] - 0s 4ms/step - loss: 0.6943 - accuracy: 0.4776 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 8/30
5/5 [==============================] - 0s 4ms/step - loss

In [33]:
predictions = model.predict(X_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

2/2 [==============================] - 0s 2ms/step


In [34]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.71
Precision: 0.64
Recall: 0.64


## Trial 3: Messing with the architecture

In [298]:
df = pd.read_csv('../exportedDataframe/a_features.csv')
dfc = read_dataframe(df)

In [299]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [300]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [301]:
tf.random.set_seed(100)# 5 -> 79%
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(14, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=50, verbose=0)

In [302]:
predictions = model.predict(X_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

2/2 [==============================] - 0s 3ms/step


In [303]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.82
Precision: 0.70
Recall: 1.00


## Trial 4

In [328]:
df = pd.read_csv('../exportedDataframe/a_features.csv')
dfc = read_dataframe(df)

In [329]:
X = dfc.drop('Labels', axis=1)
y = dfc['Labels']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [330]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [331]:
tf.random.set_seed(1)# 5 -> 79%
model = tf.keras.Sequential([
    tf.keras.layers.Dense(7, activation='relu'), # 4,5,7,1
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=50, verbose=0)

In [332]:
predictions = model.predict(X_test_scaled)
prediction_classes = [
    1 if prob > 0.5 else 0 for prob in np.ravel(predictions)
]

2/2 [==============================] - 0s 3ms/step


In [333]:
print(f'Accuracy: {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall: {recall_score(y_test, prediction_classes):.2f}')

Accuracy: 0.71
Precision: 0.75
Recall: 0.43
